In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import re

import nltk
nltk.download('punkt')

from nltk import ngrams
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
df.shape

(7613, 5)

In [8]:
df.drop('id',axis=1,inplace=True)

In [9]:
df.duplicated().sum()

np.int64(52)

In [10]:
#remove duplicates
df = df.drop_duplicates(keep='first')

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.shape

(7561, 4)

In [13]:
def clean_words(text):
    text = re.sub(r'(http.*)|#', '', text)
    tokens = word_tokenize(text)
    stops = stopwords.words('english')
    clean_tokens = [token.lower() for token in tokens if token.lower() not in stops and token.isalnum()]
    return ' '.join(clean_tokens)

In [14]:
nltk.download('punkt_tab')
df['text'] = df['text'].map(clean_words)
df.head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,keyword,location,text,target
0,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,residents asked place notified officers evacua...,1
3,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


In [15]:
df.isnull().sum()/len(df)*100

keyword      0.806772
location    33.064409
text         0.000000
target       0.000000
dtype: float64

In [16]:
df['keyword'].value_counts().sum()

np.int64(7500)

In [17]:
#lemmatizing using spacy
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy

nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):

    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])

    return lemmatized_text

df['text'] = df['text'].apply(lemmatize_text)
df.head()

You should consider upgrading via the 'D:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'd:\Practice flask\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


,keyword,location,text,target
0,NaN,NaN,deed reason earthquake may allah forgive we,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,resident ask place notify officer evacuation s...,1
3,NaN,NaN,people receive wildfire evacuation order calif...,1
4,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [18]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Predefined list of disaster-related keywords
disaster_keywords = {"earthquake", "fire", "flood", "hurricane", "storm", "evacuation",
                     "tornado", "tsunami", "disaster", "wildfire", "explosion", "drought",
                     "landslide", "eruption", "pandemic", "emergency", "crisis", "sismo", "typhoon"}

# Function to extract disaster-related keyword and proper nouns from text
def extract_keywords(text):
    # Handle NaN values in 'text' by converting to string
    if not isinstance(text, str):
        text = str(text)

    # Tokenize the text and apply POS tagging
    words = word_tokenize(text.lower())  # Convert to lowercase for case-insensitive matching
    tagged_words = pos_tag(words)  # POS tagging

    # Extract disaster-related keywords
    disaster_matches = [word for word in words if word in disaster_keywords]

    # Extract proper nouns (NNP and NNPS)
    proper_nouns = [word for word, tag in tagged_words if tag in ['NNP', 'NNPS']]

    # Combine disaster-related keywords and proper nouns
    if disaster_matches:
        return ' '.join(disaster_matches)  # Return disaster-related keyword if found
    elif proper_nouns:
        return ' '.join(proper_nouns)  # Return proper nouns if no disaster keywords are found
    return None  # Return None if no keyword found

# Apply the keyword extraction only on rows where 'keyword' is NaN
df['keyword'] = df.apply(
    lambda row: extract_keywords(row['text']) if pd.isna(row['keyword']) else row['keyword'], axis=1
)

df.head()


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Natasha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,keyword,location,text,target
0,earthquake,NaN,deed reason earthquake may allah forgive we,1
1,fire,NaN,forest fire near la ronge sask canada,1
2,evacuation,NaN,resident ask place notify officer evacuation s...,1
3,wildfire evacuation,NaN,people receive wildfire evacuation order calif...,1
4,wildfire,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [19]:
df.isnull().sum()

keyword       36
location    2500
text           0
target         0
dtype: int64

In [20]:
df.duplicated().sum()

np.int64(253)

In [21]:
# Check for duplicates based on specific columns
duplicate_rows = df[df.duplicated(subset=['text', 'keyword'])]


In [22]:
df = df.drop_duplicates()
df.duplicated().sum()


np.int64(0)

In [23]:
df.shape

(7308, 4)

In [24]:
df.head()

,keyword,location,text,target
0,earthquake,NaN,deed reason earthquake may allah forgive we,1
1,fire,NaN,forest fire near la ronge sask canada,1
2,evacuation,NaN,resident ask place notify officer evacuation s...,1
3,wildfire evacuation,NaN,people receive wildfire evacuation order calif...,1
4,wildfire,NaN,get send photo ruby alaska smoke wildfire pour...,1


In [25]:
nan_keyword_rows = df[df['keyword'].isna()]

# Display the filtered rows
print(nan_keyword_rows)

     keyword location                                               text  \
10      None      NaN                     three people die heat wave far   
14      None      NaN      damage school bus 80 multi car crash breaking   
15      None      NaN                                                man   
16      None      NaN                                         love fruit   
17      None      NaN                                      summer lovely   
18      None      NaN                                           car fast   
19      None      NaN                                    goooooooaaaaaal   
20      None      NaN                                         ridiculous   
21      None      NaN                                        london cool   
22      None      NaN                                        love skiing   
23      None      NaN                                      wonderful day   
24      None      NaN                                           looooool   
25      None

In [26]:
df['keyword'] = df['keyword'].fillna('unknown')

In [31]:
df.isnull().sum()

keyword        0
location    2345
text           0
target         0
dtype: int64

In [32]:
df.drop('location',axis=1,inplace=True)

In [33]:
df.head()

,keyword,text,target
0,earthquake,deed reason earthquake may allah forgive we,1
1,fire,forest fire near la ronge sask canada,1
2,evacuation,resident ask place notify officer evacuation s...,1
3,wildfire evacuation,people receive wildfire evacuation order calif...,1
4,wildfire,get send photo ruby alaska smoke wildfire pour...,1


In [34]:
# 1:real disaster
# 0:not real disaster

In [35]:
df.isnull().sum()

keyword    0
text       0
target     0
dtype: int64

In [37]:
#No null values found in dataframe

In [38]:
df['target'].value_counts()

target
0    4228
1    3080
Name: count, dtype: int64

In [ ]:
plt.figure(figsize=(4,4))
plt.pie(df['target'].value_counts(), labels=['not real disaster','real disaster'], autopct='%0.1f')
plt.title('Target Distribution')
plt.show()

In [ ]:
#Keyword analysis
#Unique Keywords
df['keyword'].nunique()

In [ ]:
#Most Frequent Keywords
df['keyword'].value_counts().head(20)

In [ ]:
plt.figure(figsize=(5,5))
df['keyword'].value_counts().head(20).plot(kind='bar')
plt.title('Top 10 Keywords')
plt.xlabel('Keywords')
plt.ylabel('Count')
plt.show()

In [ ]:
disaster_keywords = df[df['target'] == 1]['keyword'].value_counts().head(10)
print(disaster_keywords)

In [ ]:
non_disaster_keywords = df[df['target'] == 0]['keyword'].value_counts().head(10)
print(non_disaster_keywords)

In [ ]:
#number of words in each tweet
#df['Number_of_words'] = df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
#number of sentences in each tweet
#df['Number_of_sent'] = df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
# plt.figure(figsize=(5,4))
# sns.histplot(df['Number_of_words'], kde=True)
# plt.title('Distribution of Text Length')
# plt.show()

In [ ]:
#EDA
#1.checking percentage of each both target values
df['target'].value_counts()

In [ ]:
df.head()

In [ ]:
from gensim.models import Word2Vec

# Prepare data for Word2Vec (list of tokenized words per document)
tokenized_texts = df['text'].apply(lambda x: x.split())

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=2, workers=4)

# Check the vocabulary size
print(f"Vocabulary size: {len(w2v_model.wv)}")

In [ ]:
def get_document_vector(words, model):
    vector = np.zeros(model.vector_size)
    count = 0
    for word in words:
        if word in model.wv:
            vector += model.wv[word]
            count += 1
    return vector / count if count > 0 else vector

# Create feature matrix
df['w2v_features'] = tokenized_texts.apply(lambda x: get_document_vector(x,w2v_model))

In [ ]:
X = np.vstack(df['w2v_features'].values)
y = df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


# Define the models
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000)  # Increased max_iter for convergence
}